In [ ]:
# prompt: Genérame un archivo en formato en xlsx llamado BDHuevos, que tenga una columna llamada CPC que contenga 1000 registros, que sean valores de 6 dígitos y sean valores cerrados sin decimales, otra columna llamada CP que contenga 1000 registros, que sean valores de 6 dígitos y sean valores cerrados sin decimales, otra columna que se llame PH que contenga 1000 registros, que sean valores de 6 dígitos y sean valores cerrados sin decimales, otra columna que se llame Inflacion que contenga 1000 registros, que sean valores porcentuales de 2 dígitos, otra columna que se llame Prh y contenga 1000 registros, que contenga valores entre 500 y 1000 únicamente sin decimales, otra columna que se llame PIB que contenga 1000 registros, que sean valores porcentuales de 2 dígitos, quiero que cuando se ejecute el programa comience la descarga del archivo, quiero que este archivo tenga registros duplicados y registros faltantes. Ahora toma el archivo BDHuevos.xlsx y realiza un proceso de limpieza eliminando los valores duplicados y rellenando los valores faltantes con la moda en las columnas CPC, CP, PH, también quiero que rellenes los valores faltantes con la moda en las columnas Inflacion, PIB quiero que los conviertas a formato número sin decimales y que las cifras estén entre 1 y 10, quiero que en la columna Prh rellenes los valores faltantes con la moda, también quiero que me separes los datos en rangos de fechas desde el 2007 hasta el 2024 y que estén en formato datetime64 y finalmente quiero que me generes otro archivo llamado BDHuevos2.xlsx

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Generate data with duplicates and missing values
data = {
    'CPC': [random.randint(100000, 999999) for _ in range(1000)],
    'CP': [random.randint(100000, 999999) for _ in range(1000)],
    'PH': [random.randint(100000, 999999) for _ in range(1000)],
    'Inflacion': [random.randint(1, 10) for _ in range(1000)],
    'Prh': [random.randint(500, 1000) for _ in range(1000)],
    'PIB': [random.randint(1, 10) for _ in range(1000)],
}


#Introduce duplicates
for i in range(100):
    data['CPC'][random.randint(0,999)] = data['CPC'][random.randint(0,999)]
    data['CP'][random.randint(0,999)] = data['CP'][random.randint(0,999)]
    data['PH'][random.randint(0,999)] = data['PH'][random.randint(0,999)]
    data['Inflacion'][random.randint(0,999)] = data['Inflacion'][random.randint(0,999)]
    data['Prh'][random.randint(0,999)] = data['Prh'][random.randint(0,999)]
    data['PIB'][random.randint(0,999)] = data['PIB'][random.randint(0,999)]

#Introduce missing values
for i in range(50):
    data['CPC'][random.randint(0,999)] = np.nan
    data['CP'][random.randint(0,999)] = np.nan
    data['PH'][random.randint(0,999)] = np.nan
    data['Inflacion'][random.randint(0,999)] = np.nan
    data['Prh'][random.randint(0,999)] = np.nan
    data['PIB'][random.randint(0,999)] = np.nan

df = pd.DataFrame(data)

# Save to Excel
df.to_excel('BDHuevos.xlsx', index=False)

# Read and clean the data
df = pd.read_excel('BDHuevos.xlsx')

#Remove duplicates
df.drop_duplicates(inplace=True)


for column in ['CPC', 'CP', 'PH']:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)
    df[column] = df[column].astype(int)

for column in ['Inflacion', 'PIB']:
  mode_value = int(df[column].mode()[0])
  df[column].fillna(mode_value, inplace=True)
  df[column] = df[column].astype(int)
  df[column] = np.clip(df[column], 1, 10)

mode_prh = int(df['Prh'].mode()[0])
df['Prh'].fillna(mode_prh, inplace=True)


# Generate date range
start_date = datetime(2007, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Select random dates from range, repeat some dates
dates = np.random.choice(date_range, size = len(df))
df['Fecha'] = dates


# Save the cleaned DataFrame
df.to_excel('BDHuevos2.xlsx', index=False)

from google.colab import files
files.download('BDHuevos2.xlsx')


<ipython-input-13-a444bc1307f9>:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)
<ipython-input-13-a444bc1307f9>:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:

import pandas as pd
from prophet import Prophet
from google.colab import files

# Cargar los datos desde el archivo xlsx
df = pd.read_excel('BDHuevos2.xlsx')

# Convertir la columna 'Fecha' al formato de fecha correcto
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Crear un DataFrame para Prophet con las columnas 'ds' (fecha) y 'y' (precio de los huevos)
df_prophet = pd.DataFrame({'ds': df['Fecha'], 'y': df['Prh']})

# Inicializar el modelo Prophet
model = Prophet()

# Ajustar el modelo a los datos
model.fit(df_prophet)

# Crear un DataFrame para las fechas futuras (próximos 5 años)
future = model.make_future_dataframe(periods=1825)  # 1825 días = 5 años

# Realizar las predicciones
forecast = model.predict(future)

# Guardar las predicciones en un archivo xlsx
forecast.to_excel('predicciones_huevos.xlsx', index=False)

# Descargar el archivo
files.download('predicciones_huevos.xlsx')

# Agregar comentarios al archivo
# (Este paso se realiza después de descargar el archivo xlsx manualmente)
# En el archivo xlsx, se pueden agregar comentarios en las celdas o en el libro de trabajo
# para describir los datos y las predicciones.

# Ejemplo de comentarios dentro del código
# Este código utiliza Prophet para predecir el precio de los huevos en Colombia.
# La columna 'yhat' en el archivo de salida representa la predicción del precio de los huevos.
# 'yhat_lower' y 'yhat_upper' representan los intervalos de confianza de la predicción.

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpup8vy4wt/o6dmvnmg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpup8vy4wt/8_k_y6_x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59356', 'data', 'file=/tmp/tmpup8vy4wt/o6dmvnmg.json', 'init=/tmp/tmpup8vy4wt/8_k_y6_x.json', 'output', 'file=/tmp/tmpup8vy4wt/prophet_modelhi7zz6g0/prophet_model-20250225223656.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:36:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:36:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>